# Notebook Setup

In [11]:
import requests
import pandas as pd
import numpy as np
import json
from datetime import timedelta

# pd.set_option('display.float_format', lambda x: '%.2f' % x) # suppress scientific notation
pd.set_option('display.max_rows', 150)

# Addresses and Desired Data

In [12]:
addresses = {}

addresses['FP Main'] = '0xbc49de68bcbd164574847a7ced47e7475179c76b'
addresses['FP OPEX'] = '0xc9044dd6162106edb5ece3af8a8e8b657b612e42'
addresses['HW Phase3'] = '0x45e77E6473676aB30Ed64f3815Fea7847D02D7b8'
addresses['HW Compensation'] = '0x7Af805dcd0C18Ed276f99D83197fE7a9D37B3c54'

txtypes = {}

txtype['normal'] = 'txlist'
txtype['erc20'] = 'tokentx'
txtype['NFTs'] = 'tokennfttx'
txtype['contract_interactions'] = 'txlistinternal'


NameError: name 'txtype' is not defined

# Requests

In [ ]:
responses = []
dfs = []
for address in addresses.values():
    for action in txtypes.values():
        payload={
            'module':'account',
            'action':action,
            'address':address,
            'startblock':0,
            'endblock':'999999999',
            'sort':'asc',
            'apikey':'S53GR6DWMY1G9CY6X3U4Y9I9NKB8URMM5U'
        }
        r = requests.get('https://api.etherscan.io/api', params=payload)
        responses.append(json.loads(r.text)['result'])


for response in responses:
    dfs.append(pd.DataFrame.from_dict(response))

In [ ]:
dfs[0]


,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations
0,12362451,1620061256,0x414fa6212feca7419a1d93af7b0f55318f00a1e923b1...,2,0xc9f98cf27fb04415627d45f78acd693dce38be71f4f4...,18,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,282352,125000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,1246623,219023,1233234
1,12362770,1620065339,0x716f00f61ccfc4a854086f3688555ba9edb134291744...,3,0x3b05d7d4229f974b11fe725550d0b0d00d8356679f2c...,40,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,267798,120000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,11427160,204854,1232915
2,12362942,1620067499,0xf029ec6655770263ed70544a00faeaa72d82765d48c9...,4,0x45633dd81a0e0b2ade443d396f269cf0b1e904ece29f...,88,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,267798,95000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,8880984,204735,1232743
3,12363124,1620069897,0xa7f3704afd9e701099add4710b6e2d1d1037bfaf56bc...,5,0xa68bca33bd36638a45f45acbe0b988123036f91db787...,321,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,267798,90000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,12890613,204854,1232561
4,12363420,1620074010,0x27341748d79ea2ffd5f57f26f2b5ff236d5295d40a12...,6,0x7af980c74691cd62220f7d5c6b34e9d31cb5af4447bf...,181,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,267798,80000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,13063845,187774,1232265
5,12363426,1620074095,0x7e0a74f548f02aa684c8392af98da46a7806d6cdc7ce...,7,0x4e39cf22692954a38cb4fda3b463ef1c9bbf40d2cab1...,244,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,250039,90000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,10613991,204842,1232259
6,12363464,1620074618,0x42466a0713ea0c623683d05154d12130846e65bddb26...,8,0xf66f00016db5ecd19e6f18e3954f22e6cee60ad7c3ab...,49,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,267798,90000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,3849602,204854,1232221
7,12363470,1620074733,0x5a14ae3dd3f0b6ff772d2e85340a4f0c0a58bf122612...,9,0xc818d1cc2bdde164b68b64b9d638d116604647b37953...,92,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,267786,90000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,7898264,187881,1232215
8,12363488,1620074834,0x46704cd44067565cfc36cebd41ad14a45784e15bb122...,10,0x2c92455dc2ac249e82d761b8dd21f6ad342d2535baac...,63,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,267798,90000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,3209924,187893,1232197
9,12363499,1620074990,0x6e66712037dc39da753a1e069f1ac78646799d19000c...,11,0x96ed7656276b102f3fbe019d882405ece677dfb2c73c...,113,0x815883a8dd67b03177ec49f9f68c3834d034c483,0xbc49de68bcbd164574847a7ced47e7475179c76b,0,267798,90000000000,0,1,0x6a7612020000000000000000000000007bd29408f11d...,,7144298,204735,1232186
